# Python Web Frameworks
![image.png](attachment:image.png)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset=pd.read_csv('/content/drive/My Drive/cardio_dataset.csv').values

In [ ]:
data=dataset[:,0:7]
target=dataset[:,7]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

target=np.reshape(target, (-1,1))

scaler_data = MinMaxScaler(feature_range=(0,1))
scaler_target = MinMaxScaler()

data_scaled=scaler_data.fit_transform(data)
target_scaled=scaler_target.fit_transform(target)

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_target, test_target = train_test_split(data_scaled, target_scaled,test_size=0.2)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Dropout
import numpy as np

model = Sequential()
model.add(Dense(128, input_dim=7, activation='sigmoid',kernel_initializer='normal'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam',loss='mse',metrics=['mse','mae'])

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           650 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,941 (38.83 KB)

 Trainable params: 9,941 (38.83 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from sklearn.metrics import r2_score
import keras

class CustomCallback(keras.callbacks.Callback):

    def on_epoch_end(self,epoch,logs=None):
        predicted_result=model.predict(test_data)
        r2=r2_score(test_target,predicted_result)
        print('epoch ',epoch,'- r2 score:',r2)

[Full details at keras official site](https://www.tensorflow.org/guide/keras/custom_callback)

In [ ]:
from keras.callbacks import ModelCheckpoint

# Change the file extension from .model to .keras
checkpoint = ModelCheckpoint('models/model-{epoch:03d}.keras',monitor='val_loss',save_best_only=True,mode='auto')

In [ ]:
history=model.fit(train_data,train_target,epochs=200,validation_data=(test_data,test_target),callbacks=[checkpoint,CustomCallback()])

Epoch 1/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
epoch  0 - r2 score: 0.02036219309451437
167/167 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0655 - mae: 0.1942 - mse: 0.0655 - val_loss: 0.0253 - val_mae: 0.1232 - val_mse: 0.0253
Epoch 2/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
epoch  1 - r2 score: 0.031111662502798287
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0284 - mae: 0.1280 - mse: 0.0284 - val_loss: 0.0250 - val_mae: 0.1200 - val_mse: 0.0250
Epoch 3/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
epoch  2 - r2 score: 0.04585343314564405
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0247 - mae: 0.1195 - mse: 0.0247 - val_loss: 0.0246 - val_mae: 0.1211 - val_mse: 0.0246
Epoch 4/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
epoch  3 - r2 score: 0.049760634600779174
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0249 - mae: 0.1188 - mse: 0.0249 - val_loss: 0.0245 - val_mae: 0.1110 - val_mse: 0.0245
Epoch 5/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
epoch  4 - r2 score: 0

In [ ]:
from matplotlib import pyplot as plt

plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.xlabel('# epochs')
plt.ylabel('loss')
plt.show()

In [ ]:
print('actual:',test_target[:10].T)
print('predicted:',predicted_result[:10].T)

In [ ]:
print('actual inverse scaled:',scaler_target.inverse_transform(test_target[:10]).T)
print('predicted inverse scaled:',scaler_target.inverse_transform(predicted_result[:10]).T)

# Save all the models

In [ ]:
import joblib

joblib.dump(scaler_data,'scaler_data.sav')
joblib.dump(scaler_target,'scaler_target.sav')